# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [36]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
#         pass # put your code here
        kernel = (kernel + 1) ** degree
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [257]:
import itertools
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import cvxopt

In [258]:
def polynomial_kernel(x, y, degree=3, coef0=1):
    return (np.dot(x, y) + coef0) ** degree

def choose_set_for_label(data_set, labels, label):
    return data_set[labels == label, :]

def get_labels_count(data_set, labels, label):
    return len(data_set[labels == label])

In [259]:
def build_kernel(data_set, kernel_type='linear', degree=3, coef0=1):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'polynomial':
        kernel = polynomial_kernel(data_set, data_set.T, degree, coef0)
    elif kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T) + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

In [260]:
def train(train_data_set, train_labels, kernel_type='linear', degree=3, coef0=1, C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type, degree=degree, coef0=coef0)
    objects_count = len(train_data_set)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b), options={'show_progress': True})

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number


In [261]:
def train_multiclass(train_data_set, train_labels, kernel_type='linear', degree=3, coef0=1, C=10, threshold=1e-5):
    unique_labels = np.unique(train_labels)
    classifiers = {}
    for i in range(len(unique_labels)):
        for j in range(i+1, len(unique_labels)):
            label_i, label_j = unique_labels[i], unique_labels[j]
            binary_train_labels = train_labels[(train_labels == label_i) | (train_labels == label_j)]
            binary_train_data_set = train_data_set[(train_labels == label_i) | (train_labels == label_j)]
            binary_train_labels[binary_train_labels == label_i] = -1
            binary_train_labels[binary_train_labels == label_j] = 1
            classifiers[(label_i, label_j)] = train(binary_train_data_set, binary_train_labels, kernel_type, degree, coef0, C, threshold)
    return classifiers

In [262]:
def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [263]:
iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, labels, test_size=0.2, random_state=42)

classifiers = train_multiclass(train_data_set, train_labels, kernel_type='polynomial', degree=3, coef0=1, C=10, threshold=1e-5)

predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))
print("Multiclass C-SVM accuracy with polynomial kernel:", accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  1.8337e-01 -9.3193e+02  2e+03  2e-01  4e-11
 1:  9.0822e-02 -2.7948e+01  5e+01  3e-03  3e-11
 2:  4.3770e-02 -1.7612e+00  3e+00  2e-04  3e-12
 3:  1.5191e-03 -2.3259e-01  4e-01  2e-05  4e-13
 4:  4.4677e-04 -2.3595e-02  4e-02  2e-06  6e-14
 5:  4.1068e-04 -7.5803e-04  1e-03  2e-08  1e-14
 6:  4.8559e-05 -1.0554e-04  2e-04  2e-16  3e-15
 7: -2.7201e-06 -2.3642e-05  2e-05  2e-16  1e-15
 8: -7.8438e-06 -1.2629e-05  5e-06  2e-16  6e-16
 9: -9.9894e-06 -1.3164e-05  3e-06  3e-16  3e-16
10: -1.1231e-05 -1.1826e-05  6e-07  2e-16  4e-16
11: -1.1530e-05 -1.1554e-05  2e-08  2e-16  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.3236e-01 -9.0995e+02  2e+03  2e-01  7e-11
 1:  6.6251e-02 -2.9230e+01  5e+01  3e-03  6e-11
 2:  3.2783e-02 -1.2928e+00  2e+00  1e-04  8e-12
 3:  1.9916e-03 -1.6535e-01  3e-01  1e-05  9e-13
 4:  2.5682e-04 -1.7793e-02  3e-02  1e-06  1e-13
 5:  2.4333e-04 -4.8625e-04  8e-04  1e-08  1e-1